In [1]:
import os, sys
import argparse
import ast
import pandas as pd

import advertools as adv

In [2]:
df = pd.read_csv('sorted_sections.csv')
df

,Unnamed: 0.1,Unnamed: 0,ecli,date,inhoudsindicatie,procesverloop,overwegingen,beslissing,sections,organized
0,0,0,ECLI:NL:CBB:2022:1,2022-01-11,"\n\nArtikel 2:3, eerste lid, van de Algemene w...",Procesverloop Bij besluit van 2 oktober 2020 (...,Overwegingen 1.1 Appellant heeft op 10 mei 202...,Beslissing Het College: - verklaart het beroep...,['1.1 Appellant heeft op 10 mei 2020 een aanvr...,{1: '1.1 Appellant heeft op 10 mei 2020 een aa...
1,1,1,ECLI:NL:CBB:2022:10,2022-01-18,\n\nMond - en klauwzeer. Medio maart 2001 is i...,Procesverloop Bij besluit van 29 maart 2001 (h...,Overwegingen Feiten en omstandigheden 1. Het C...,Beslissing Het College: verklaart het beroep g...,['1. Het College gaat uit van de volgende feit...,{1: '1. Het College gaat uit van de volgende f...
2,2,2,ECLI:NL:CBB:2022:100,2022-03-08,\n\n-\tBeleidsregel tegemoetkoming ondernemers...,Procesverloop Bij besluit van 3 februari 2021 ...,Overwegingen 1. Het wettelijk kader is opgenom...,Beslissing Het College: - verklaart het beroep...,['1. Het wettelijk kader is opgenomen in de bi...,{1: '1. Het wettelijk kader is opgenomen in de...
3,3,3,ECLI:NL:CBB:2022:101,2022-03-08,"\nWarenwet, hoger beroep, Verordening 853/2004...",Procesverloop in hoger beroep De minister heef...,Grondslag van het geschil 1.1 De rechtbank hee...,Beslissing Het College bevestigt de aangevalle...,['1.1 De rechtbank heeft het beroep van [naam ...,{1: '1.1 De rechtbank heeft het beroep van [na...
4,4,4,ECLI:NL:CBB:2022:102,2022-03-08,\nRegeling garanties van oorsprong en certific...,Procesverloop Bij afzonderlijke besluiten van ...,Overwegingen Inleiding 1.1 Appellanten exploit...,Beslissing Het College: verklaart de beroepen ...,['1.1 Appellanten exploiteren een eigen biover...,{1: '1.1 Appellanten exploiteren een eigen bio...
...,...,...,...,...,...,...,...,...,...,...
95,95,95,ECLI:NL:CBB:2022:206,2022-05-03,\nISDE-subsidie voor warmtepompen. Aanvragen a...,Procesverloop Bij 32 afzonderlijke besluiten v...,Overwegingen In de periode van 12 maart 2019 t...,Beslissing Het College verklaart het beroep on...,['4. Verweerder meent dat het beroep van appel...,{4: '4. Verweerder meent dat het beroep van ap...
96,96,96,ECLI:NL:CBB:2022:207,2022-05-03,\nSDE-subsidie voor co-vergistingsinstallaties...,Procesverloop Appellanten hebben beroep ingest...,Overwegingen 1.1 Verweerder heeft aan appellan...,Beslissing Het College verklaart de beroepen o...,['1.1 Verweerder heeft aan appellanten subsidi...,{1: '1.1 Verweerder heeft aan appellanten subs...
97,97,97,ECLI:NL:CBB:2022:208,2022-05-03,\nVerzoek om schadevergoeding wordt afgewezen....,Procesverloop Verzoeker heeft op 4 december 20...,Overwegingen Relevante bepalingen 1.\tOp grond...,Beslissing Het College wijst het verzoek tot s...,['1.\tOp grond van artikel8:88 van de Awb is d...,{1: '1.\tOp grond van artikel8:88 van de Awb i...
98,98,98,ECLI:NL:CBB:2022:209,2022-05-03,\n\nFosfaatrechten – verzoek schadevergoeding\...,Procesverloop Op 29 oktober 2020 heeft verzoek...,Overwegingen Relevante bepalingen 1.1 Op grond...,Beslissing Het College: veroordeelt verweerder...,"['1.1 Op grond van artikel8:88, eerste lid, aa...","{1: '1.1 Op grond van artikel8:88, eerste lid,..."


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 3)
stopwords_dutch = list(adv.stopwords['dutch'])
df['organized'] = df.organized.apply(ast.literal_eval)
df.organized


0     {1: '1.1 Appellant heeft op 10 mei 2020 een aa...
1     {1: '1. Het College gaat uit van de volgende f...
2     {1: '1. Het wettelijk kader is opgenomen in de...
3     {1: '1.1 De rechtbank heeft het beroep van [na...
4     {1: '1.1 Appellanten exploiteren een eigen bio...
                            ...                        
95    {4: '4. Verweerder meent dat het beroep van ap...
96    {1: '1.1 Verweerder heeft aan appellanten subs...
97    {1: '1.	Op grond van artikel8:88 van de Awb is...
98    {1: '1.1 Op grond van artikel8:88, eerste lid,...
99    {1: '1. Appellant heeft op 4 mei 2020 een aanv...
Name: organized, Length: 100, dtype: object

In [4]:
docs = [value for sublist in df['organized'].apply(lambda d: list(d.values())) for value in sublist]

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stopwords_dutch).fit(docs)
candidates = count.get_feature_names_out()
#print(candidates)

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert/distilbert-base-multilingual-cased')
doc_embedding = model.encode([docs])
candidate_embeddings = model.encode(candidates)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [ ]:
keywords

In [ ]:
import numpy as np
import itertools

def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)